# ACS AWS Training: Day 3
### Nathan Miles
06/08/2020

## Outline
- Review of last time

- EC2 Launch Template

- AWS setup scripts


- Example
    - Download HST data from the MAST public cache in S3 to EC2
    - Compute aperture photometry of all sources identified in each image using a 3 pixel aperture
    - For each image,
        - Save a catalog of the sources found for each chip
        - Save a plot showing all sources 
    - Upload the results from the analysis to S3
    - Downloading the results from S3 to your computer

- EC2 Amazon Machine Image (AMI)

## Day 2 Review
- Programmatic AWS Access
    - Requires use the of an access key id and its corresponding secret access key
        - e.g. Downloading data from S3 to an EC2 instance or your local machine
    - The access key id and secrect access key are stored in the credentials file
        - `~/.aws/credentials`
- The credentials file is easiest to configure using the AWS CLI (<a href="https://nmiles2718.github.io/presentations/acs_aws_day2/day2.html#/4">Day 2, Slide 4</a>)
- EC2 instances are secured with an ssh key-pair and via Security Groups
- Launching an EC2 instance (<a href="https://nmiles2718.github.io/presentations/acs_aws_day2/day2.html#/7">Day2, Slide 7</a>)
- Connecting to an EC2 instance (<a href="https://nmiles2718.github.io/presentations/acs_aws_day2/day2.html#/8">Day2, Slide 8</a>)

## EC2 Launch Template
- Designed to reduce the overhead in repeatedly using the same EC2 instance configuration
- For the typical use case in astronomy, the following parameters are the most important,
    - AMI
    - Instance type
    - Storage
    - ssh key pair
    - Security group
- All the parameters you can specify when launching an EC2 instance from the AWS Management Console can be configured in a launch template

## Creating an EC2 Launch Template
- Login to the AWS management console
- Navigate to the EC2 dashboard
- In the toolbar on the left,
    - Instances > Launch Templates 
- Click "Create Launch Template"
    - Give your template a name and description
    - AMI: `aws_acs_day3`
        - Custom Linux AMI with Minicond3 and additional software pre-installed
    - Instance type: `c5.xlarge`
        - 4 cores, 8 GiB RAM, $0.17/hour
    - Key pair: Select your key pair form the dropdown menu
    - Security groups: Select your security group

## Launching an EC2 Instance via Launch Templates
- Navigate to the dashboard for Launch Templates
- Select the launch template you just created
    - Copy the Launch Template ID
        - e.g., lt-01e33604ade7b0c57
- Copy and paste the following code in an `ipython` session on your local machine
    - Update the `template_id` with the Launch Template ID for the template you just created


```python
import boto3

template_id='YOUR_TEMPLATE_ID'
s = boto3.Session(region_name='us-east-1', profile_name=None)
ec2 = s.resource('ec2')
LaunchTemplate = {'LaunchTemplateId': template_id}
instances = ec2.create_instances(LaunchTemplate=LaunchTemplate,
                     MaxCount=1,
                     MinCount=1)

instance = instances[0]

print('Launched EC2 instance {}'.format(instance.id))
instance.wait_until_running()
instance.load()
print(f"EC2 DNS: {instance.public_dns_name}")

```

## AWS EC2 Setup Scripts: `setup_ec2.sh`
- Use `scp` to securely copy over configuration info via ssh
- Copy over the entire `~/.aws/` directory
- Copy over github ssh keys
- Login to the machine

``` shell
#!/usr/bin/env bash

if [[ $# -lt 2 ]] ; then
    echo "Not enough arguments supplied"
    echo "Required Positional Arguments:"
    echo "1. EC2 public DNS"
    echo "2. EC2 private ssh key"
    exit 1
fi

# Copy credentials to EC2 instanace
ec2_ip=$1  # first arg is IP address of instance
keyname=$2
aws_dir=~/.aws/
ssh_credentials=~/.ssh/
aws_ssh_key=$aws_dir$keyname

# Copy files credentials to AWS
scp -ri "$aws_ssh_key" $aws_dir ec2-user@$ec2_ip:~
scp -ri "$aws_ssh_key" $ssh_credentials ec2-user@$ec2_ip:~
scp -ri "$aws_ssh_key" ~/aws_setup/install_miniconda3_linux.sh ec2-user@$ec2_ip:~

# log-in to the instance
ssh -i "$aws_ssh_key" ec2-user@$ec2_ip
```

## AWS EC2 Setup Scripts: `login.sh`

```shell
#!/usr/bin/env bash

if [[ $# -lt 2 ]] ; then
    echo "Not enough arguments supplied"
    echo "Required Positional Arguments:"
    echo "1. EC2 public DNS"
    echo "2. EC2 private ssh key"
    exit 1
fi

ec2_ip=$1  
keyname=$2
aws_dir=~/.aws/
aws_ssh_key=$aws_dir$keyname

# log-in to the instance
ssh -i "$aws_ssh_key" ec2-user@$ec2_ip
```

## Example
- In the following example you will,
    - Create an S3 bucket
    - Download a series of CRJ/CRC images
    - Compute magnitudes for all sources in each image
        - Aperture sum is computed with 3 pixel aperture
        - Background is computed using annulus with $R_{in}=14$, $R_{out}=17$
        - Catalogs are stored in chip dependent files
    - Generate a plot with both chips and the sources identified on each chip overlaid
    - Upload the results to S3

## Example
Step 1) Use the `setup_ec2.sh` script to copy over your AWS access key info
 
  - The custom AMI includes the following,
      - Miniconda3 pre-installed
      - The following Python packages
           - `astropy`, `astroquery`, `numpy`, `matplotlib`, and `photutils`
      - 3 python modules to be used in the example
           - `download.py`
           - `photometry.py`
           - `transfer_output.py`
 
Step 2) Download CRJ/CRC images from Proposal 14507 (External CTE Program)
```console 
python download.py -proposal_id 14507
```

## Example

Step 3) Run the photometry pipeline (will take about 8 minutes)
```console
python photometry.py
```

Step 4) Transfer the results to S3
```console
python transfer_output.py -bucket_name <YOUR_BUCKET> -folder_name <YOUR_FOLDER> 
```

## Example
Step 5) Download the results from S3 to your laptop
```python
import os
import boto3
try:
    os.mkdir('./aws_results_day3')
except FileExistsError:
    print('Directory already exists')
session = boto3.Session()
client = session.resource('s3', region_name='us-east-1')
bucket = client.Bucket('aws-acs-tutorial-day3')
for f in list(bucket.objects.all()):
    bucket.download_file(f.key, f'aws_results_day3/{os.path.basename(f.key)}')
```

## EC2 AMI
- The base operating system for an EC2 instance
- Each AMI has the following,
    - A template for the root volume of the instance
        - EBS-backed uses an EBS volume for root
        - instance-store-backed uses an template stored in S3 
        - <a href="https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/RootDeviceStorage.html">More info on EC2 root device volume</a>
    - Launch permissions to control which AWS accounts can use the AMI to launch the instance
        - Public or private
    - A block device mapping to specify any additional volumes to attach to the instance when it is launched


## Creating an EBS-backed EC2 AMI
- Launch an EC2 instance
- Install your favorite software
- In the EC2 dashboard of AWS Management Console,
    - Navigate to the running instances
    - Select your instance
    - Actions > Image > Create Image